## CURRENT NOTES

(03/23/20):

~I've updated my bacteria class and its functions to match your version for mesa

~I've added a carrying capacity function to be applied IMMEDIATELY AFTER reproduce function

(02/24/20):

~ I've added the reproduction function

~ It can be used in a .apply() manner

(02/12/20):

~ There are now functions for gene transfer and two different ways of displaying data

~ Still need work on bacteria reproduction function. A few decisions need to be made (likely as a group)


(02/10/20):

~ There are no functions yet

~ Let's work this week to implement the determinisitic model sent to us

---

- This code establishes the bacteria class to be used in the agent-based model

- The final model does not have to be done in this notebook

- This should just be the building ground for the individuals and their functions in the ABM

In [1]:
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np 
import math
import pylab as pl
import math
import random as rd 
import copy
from mesa import Agent
from mesa import space

In [2]:
##initializing the bacteria class
  
class bacteria(Agent):
    def __init__(self, unique_id,model,state=0,age=0,volume=0.1,tax='bac',gene_arg='mce',mge="afg",gene_pres=0,mge_pres=0):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.model = model
        self.state = state #{0:susceptible, 1:resistant, 2:acquired resistant}
        self.age=age
        self.volume = volume
        self.tax = tax
        self.gene_arg = gene_arg
        self.gene_pres = gene_pres
        self.mge = mge
        self.mge_pres = mge_pres
        
    def display(self):
        #will display all features of bacteria
        print("Unique ID: ",self.unique_id)
        print("Model: ",self.model)
        print("State: ",self.state)
        print("Age: ",self.age)
        print("Volume: ",self.volume)
        print("Taxonomy: ",self.tax)
        print("ARG Presence: ",self.gene_pres)
        print("ARG: ",self.gene_arg)
        print("MGE Presence: ",self.mge_pres)
        print("MGE: ",self.mge)
        print("")
        
    def build_series(self):
        row = pd.Series([self.unique_id,self.model,self.state,self.age,self.volume,self.tax,self.gene_pres,self.gene_arg,self.mge_pres,self.mge],
                        index=["unique_id","model","state","age","volume","taxonomy","gene_pres","gene","mge_pres","mge"])
        return row

In [3]:
##This function changes the state to acuired resistance
##Meant to be used in a .apply or map() manner
def gene_transferred(x):
    x.state = 2
    return x

In [4]:
##colony is a list of bacteria class objects
##This function reassigns the states of susceptible bacteria based on the horizontal gene transfer rates

##WHEN DONE TESTING BE SURE TO REMOVE PRINT() LINES

def horizontal_gene_transfer(colony,acq_trans_rate=0.0001,res_trans_rate=0.0001):
    
    ##Number of newly resistant bacteria from acquired resistance bacteria
    new_acq = acq_trans_rate*len([x for x in colony if x.state == 2])
    
    ##Number of newly resistant bacteria from originally resistant bacteria
    new_res = res_trans_rate*len([x for x in colony if x.state == 1])
    
    ##randomly choose which bacteria will be new resistant bacteria
    print(new_acq+new_res)
    changing = rd.sample([x for x in colony if x.state == 0],round(new_acq+new_res))
    print(changing)
    list(map(gene_transferred,[x for x in changing if x.state == 0])) #This will globally change state; don't worry about it being a list
    
    
    return #Nothing to return since changes were global
    

In [5]:
def group_df(bac_list):
    ##bac_list must be a list of bacteria objects
    ##this can be a full colony or any specified subgroup of a colony
    ##returns a dataframe of a copy of bacteria object values (NOT OBJECTS)
    
    rows=[]
    for i in bac_list:
        rows.append(i.build_series())

    col_df = pd.DataFrame(rows)
    

    return col_df

In [6]:
##Creates a number of copies depending on state reproduction rate of a given bacteria object
##Returns a list of offspring bacteria objects

##Capable of being used in a .apply() manner

def reproduce(bac,sus_growth = 2,res_growth = 1.1,acq_growth = 1.1):
    ##Creates reproduced bacteria based on given bacteria
    
    offspring = []
    
    if bac.state == 0:
        rep_rate = sus_growth
        
    elif bac.state == 1:
        rep_rate = res_growth
        
    elif bac.state == 2:
        rep_rate = acq_growth
        
    else:
        raise Exception("Unknown Bacteria Object state")
            
            
    for i in range(round(rep_rate)):
        offsp = copy.copy(bac)
        offsp.age = 0
        offspring.append(offsp)
            
    
    return offspring

In [7]:
##Removes bacteria from colony based on carrying capacity
##Assumes homogeneity in removal

##MUST BE APPLIED IMMEDIATELY AFTER REPRODUCE FUNCTION
def carry_cap(colony,sus_cap,res_cap,acq_cap):
     
    
    sus_pop = [x for x in colony if x.state == 0]
    sus_pop = rd.sample(sus_pop, round(len(sus_pop)/sus_cap))
    colony = [x for x in colony if x not in sus_pop]
    
    res_pop = [x for x in colony if x.state == 1]
    res_pop = rd.sample(res_pop, round(len(res_pop)/res_cap))
    colony = [x for x in colony if x not in res_pop]
    
    acq_pop = [x for x in colony if x.state == 2]
    acq_pop = rd.sample(acq_pop, round(len(acq_pop)/acq_cap))
    colony = [x for x in colony if x not in acq_pop]
    

    
    return colony

## Testing Zone

In [31]:
col = [bacteria("ID","model",1,0.3,0.75,'streptococcus',1,'ArB1',0,0),bacteria("ID","model",0,0.3,0.75,'sdfsa',1,'ArB1',0,0),bacteria("ID","model",1,0.3,0.75,'streptococcus',1,'ArB1',0,0)]
for i in range(3):
    col.append(bacteria("ID","model",0,0.3,0.75,'dafsdva',0,0,0,0))
    
for i in range(3):
    col.append(bacteria("ID","model",2,0.3,0.75,'dafsdva',0,0,0,0))
    
for i in range(5):
    col.append(bacteria("ID","model",1,0.3,0.75,'dafsdva',0,0,0,0))

In [27]:
x = bacteria("ID","model",1,0.3,0.75,'dafsdva',0,0,0,0)
x.display()

Unique ID:  ID
Model:  model
State:  1
Age:  0.3
Volume:  0.75
Taxonomy:  dafsdva
ARG Presence:  0
ARG:  0
MGE Presence:  0
MGE:  0



In [29]:
len(col)

14

In [30]:
len([x for x in col if x.state == 2])

3

In [31]:
horizontal_gene_transfer(col,res_trans_rate=.3,acq_trans_rate=.6)

3.9
[<__main__.bacteria object at 0x000001F94C459C88>, <__main__.bacteria object at 0x000001F94C459C50>, <__main__.bacteria object at 0x000001F94C55F7B8>, <__main__.bacteria object at 0x000001F94C55F828>]


In [32]:
len([x for x in col if x.state == 2])

7

In [33]:
group_df([x for x in col if x.state == 2])

,unique_id,model,state,age,volume,taxonomy,gene_pres,gene,mge_pres,mge
0,ID,model,2,0.3,0.75,sdfsa,0,1,0,ArB1
1,ID,model,2,0.3,0.75,dafsdva,0,0,0,0
2,ID,model,2,0.3,0.75,dafsdva,0,0,0,0
3,ID,model,2,0.3,0.75,dafsdva,0,0,0,0
4,ID,model,2,0.3,0.75,dafsdva,0,0,0,0
5,ID,model,2,0.3,0.75,dafsdva,0,0,0,0
6,ID,model,2,0.3,0.75,dafsdva,0,0,0,0


In [25]:
test = copy.copy(col[0])
test.state = 1
test.display()

State:  1
Taxonomy:  streptococcus
ARG Presence:  1
ARG:  ArB1
MGE Presence:  0
MGE:  0
Biome:  ocean



In [27]:
reproduce(test, res_growth=5)

In [15]:
len(col)

14

In [37]:
acq_pop = [x for x in col if x.state == 2]
acq_pop = rd.sample(acq_pop, round(len(acq_pop)/2))
col = [x for x in col if x not in acq_pop]
len(col)

12